# Property Value Trends in NH
## Combining and Cleaning Data

In [158]:
import pandas as pd
import re
import numpy as np

## Read and Clean 2011 Data

Property tax data sourced from the [NH Dept. of Revenue Administration](https://www.revenue.nh.gov/mun-prop/municipal/documents/2009-local.pdf). I exported the PDF to a csv file. Some cleanup of the file was necessary.

In [140]:
df_2011 = pd.read_excel('../Raw Data/2011-local.xlsx', engine="openpyxl")
df_2011 = df_2011.drop(['P', 'Date\nIssued', 'Unnamed: 10'], axis=1)
df_2011.columns = ['Town', 'Valuation', 'Town Tax', 'Local Ed. Tax', 'State Ed. Tax', 'County Tax', 'Total Tax', 'Total Commitment']

#Revise town name formatting to match 2020 data
df_2011['Town'] = df_2011['Town'].str.title()
df_2011['Town'] = df_2011['Town'].str.replace("'S", "'s")

df_2011 = df_2011.set_index('Town')

#Rectify a few towns who's names are different between the 2011 and 2020 datasets
df_2011 = df_2011.rename(index={'Thom. & Mes.':'Thom. & Mes. Purchase',
                                'Low & Burbank Gr': "Low & Burbank's Grant",
                                'Atk. & Gilmanton': 'Atkinson & Gilmanton Academy Grant'})

df_2011.head()

,Valuation,Town Tax,Local Ed. Tax,State Ed. Tax,County Tax,Total Tax,Total Commitment
Town,,,,,,,
Acworth,94713062,6.83,8.65,2.51,3.20,21.19,1996801
Albany,103641581,3.05,4.15,2.50,1.06,10.76,1089760
Alexandria,200605414,5.57,12.70,2.60,1.50,22.37,4403307
Allenstown,267129444,7.93,16.30,2.34,2.73,29.30,7680063
Alstead,178908827,4.91,10.74,2.38,2.91,20.94,3734318


### Adjust valuation for inflation. 
`$1.00 (2011) = $1.17 (2020)`, according the [U.S. Bureau of Labor Statistics](https://data.bls.gov/cgi-bin/cpicalc.pl). First, we need to clean up the valuation data.

In [121]:
def clean_int(x):
    
    x = str(x)
    x = x.replace(',','')
    x = x.replace('*','')
    x = x.replace('$','')
    x = x.strip()
    x = re.search('([0-9]*)', x)[0]        
   
    try:
        return int(x)
    except:
        return -999

In [141]:
df_2011.Valuation = df_2011.Valuation.apply(clean_int).astype('int')
df_2011.Valuation = df_2011.Valuation * 1.17
df_2011['Total Commitment'] = df_2011['Valuation'] * df_2011['Total Tax']

df_2011[['Valuation', 'Total Commitment']] = df_2011[['Valuation', 'Total Commitment']].astype('int')

In [142]:
df_2011

,Valuation,Town Tax,Local Ed. Tax,State Ed. Tax,County Tax,Total Tax,Total Commitment
Town,,,,,,,
Acworth,110814282,6.83,8.65,2.51,3.20,21.19,2348154647
Albany,121260649,3.05,4.15,2.50,1.06,10.76,1304764591
Alexandria,234708334,5.57,12.70,2.60,1.50,22.37,5250425440
Allenstown,312541449,7.93,16.30,2.34,2.73,29.30,9157464469
Alstead,209323327,4.91,10.74,2.38,2.91,20.94,4383230479
...,...,...,...,...,...,...,...
Winchester,330630545,7.18,17.50,2.11,3.24,30.03,9928835287
Windham,2369279682,3.79,15.77,2.46,1.06,23.08,54682975081
Windsor,26781515,0.91,8.81,3.57,1.19,14.48,387796341


## Read and clean 2020 data

In [133]:
usecols=['Municipality',
         'Valuation w/ Utils',
         'Municipal',
         'County',
         'State Ed.',
         'Local Ed.', 
         'Total Rate', 
         'Total Commitment']

df_2020 = pd.read_excel('../Raw Data/20-tax-rates.xlsx', 
                        engine='openpyxl',
                        usecols=usecols,
                        skiprows=5)

# Match w/2011 dataframe
df_2020.columns = ['Town',
                   'Valuation',
                   'Town Tax',
                   'County Tax',
                   'State Ed. Tax',
                   'Local Ed. Tax',
                   'Total Tax', 
                   'Total Commitment']

#Edit names for unincorporate places to match 2011 data
df_2020['Town'] = df_2020['Town'].str.replace(' \(U\)', '', regex=True)

df_2020 = df_2020.set_index('Town')

df_2020

,Valuation,Town Tax,County Tax,State Ed. Tax,Local Ed. Tax,Total Tax,Total Commitment
Town,,,,,,,
Acworth,100103637,9.66,2.60,1.97,15.14,29.37,2921681
Albany,130141621,2.20,1.08,1.71,7.06,12.05,1546965
Alexandria,198181477,9.32,1.99,2.20,14.41,27.92,5460145
Allenstown,299434087,9.62,2.76,2.03,14.89,29.30,8652469
Alstead,193481509,5.51,3.46,1.92,13.53,24.42,4692133
...,...,...,...,...,...,...,...
Winchester,333847858,7.09,3.15,1.68,16.85,28.77,9436256
Windham,3059617070,2.99,0.85,1.84,13.45,19.13,58209101
Windsor,28777018,0.42,0.94,2.03,6.42,9.81,276423


## Combine dataframes

In [201]:
df = df_2011.merge(df_2020, left_index=True, right_index=True, suffixes=(' 2011', ' 2020'), how='outer')
df

,Valuation 2011,Town Tax 2011,Local Ed. Tax 2011,State Ed. Tax 2011,County Tax 2011,Total Tax 2011,Total Commitment 2011,Valuation 2020,Town Tax 2020,County Tax 2020,State Ed. Tax 2020,Local Ed. Tax 2020,Total Tax 2020,Total Commitment 2020
Town,,,,,,,,,,,,,,
Acworth,110814282,6.83,8.65,2.51,3.20,21.19,2348154647,100103637,9.66,2.60,1.97,15.14,29.37,2921681
Albany,121260649,3.05,4.15,2.50,1.06,10.76,1304764591,130141621,2.20,1.08,1.71,7.06,12.05,1546965
Alexandria,234708334,5.57,12.70,2.60,1.50,22.37,5250425440,198181477,9.32,1.99,2.20,14.41,27.92,5460145
Allenstown,312541449,7.93,16.30,2.34,2.73,29.30,9157464469,299434087,9.62,2.76,2.03,14.89,29.30,8652469
Alstead,209323327,4.91,10.74,2.38,2.91,20.94,4383230479,193481509,5.51,3.46,1.92,13.53,24.42,4692133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Winchester,330630545,7.18,17.50,2.11,3.24,30.03,9928835287,333847858,7.09,3.15,1.68,16.85,28.77,9436256
Windham,2369279682,3.79,15.77,2.46,1.06,23.08,54682975081,3059617070,2.99,0.85,1.84,13.45,19.13,58209101
Windsor,26781515,0.91,8.81,3.57,1.19,14.48,387796341,28777018,0.42,0.94,2.03,6.42,9.81,276423


## Create rows for NH Totals

Summing valution and total tax committments and taking a weighted average of tax rates.

In [208]:
nh_avg_tax_rates_2011 = df.iloc[:, 1:6].apply(lambda x: np.average(x, weights=df['Valuation 2011']))
nh_avg_tax_rates_2011

Town Tax 2011          6.199296
Local Ed. Tax 2011     9.995924
State Ed. Tax 2011     2.420499
County Tax 2011        1.606652
Total Tax 2011        20.222371
dtype: float64

In [211]:
nh_avg_tax_rates_2020 = df.iloc[:, 8:-1].apply(lambda x: np.average(x, weights=df['Valuation 2020']))
nh_avg_tax_rates_2020

Town Tax 2020          6.400930
County Tax 2020        1.577626
State Ed. Tax 2020     1.981175
Local Ed. Tax 2020    11.213627
Total Tax 2020        21.173358
dtype: float64

In [226]:
# Initialize
df.loc['NH'] = np.nan

df.loc['NH', ['Valuation 2011', 'Valuation 2020']] = df[['Valuation 2011', 'Valuation 2020']].sum()

df.loc['NH', nh_avg_tax_rates_2011.index] = nh_avg_tax_rates_2011
df.loc['NH', nh_avg_tax_rates_2020.index] = nh_avg_tax_rates_2020

df.loc['NH', ['Total Commitment 2011', 'Total Commitment 2020']] = df[['Total Commitment 2011', 'Total Commitment 2020']].sum()


df.loc['NH']

Valuation 2011           1.837661e+11
Town Tax 2011            6.199296e+00
Local Ed. Tax 2011       9.995924e+00
State Ed. Tax 2011       2.420499e+00
County Tax 2011          1.606652e+00
Total Tax 2011           2.022237e+01
Total Commitment 2011    3.716187e+12
Valuation 2020           1.917272e+11
Town Tax 2020            6.400930e+00
County Tax 2020          1.577626e+00
State Ed. Tax 2020       1.981175e+00
Local Ed. Tax 2020       1.121363e+01
Total Tax 2020           2.117336e+01
Total Commitment 2020    4.032124e+09
Name: NH, dtype: float64

## Export to a CSV

In [227]:
df.to_csv('../Cleaned Data/NH_Property_Tax_Rates_2011-20.csv')

## Create dataframe filtered for SWRPC communities only

In [248]:
towns_url = 'https://raw.githubusercontent.com/toddhorner/swrpc_data/master/Utility%20Datasets/SWRPC_munis.csv'
swrpc_towns = pd.read_csv(towns_url, usecols=['TOWN'], squeeze=True)
swrpc_towns = swrpc_towns.str.strip().tolist()

df_swrpc = df.loc[swrpc_towns]

df_swrpc = df_swrpc.sort_index()

In [249]:
df_swrpc.loc['NH'] = df.loc['NH']

In [250]:
df_swrpc

,Valuation 2011,Town Tax 2011,Local Ed. Tax 2011,State Ed. Tax 2011,County Tax 2011,Total Tax 2011,Total Commitment 2011,Valuation 2020,Town Tax 2020,County Tax 2020,State Ed. Tax 2020,Local Ed. Tax 2020,Total Tax 2020,Total Commitment 2020
Town,,,,,,,,,,,,,,
Alstead,2.093233e+08,4.910000,10.740000,2.380000,2.910000,20.940000,4.383230e+09,1.934815e+08,5.51000,3.460000,1.920000,13.530000,24.420000,4.692133e+06
Antrim,2.947224e+08,10.930000,8.640000,2.450000,1.120000,23.140000,6.819877e+09,2.548074e+08,10.57000,1.130000,2.050000,14.800000,28.550000,7.175969e+06
Bennington,1.327141e+08,9.920000,9.910000,2.330000,1.040000,23.200000,3.078968e+09,1.286521e+08,10.99000,1.020000,1.860000,15.520000,29.390000,3.731435e+06
Chesterfield,6.569393e+08,3.420000,8.390000,2.220000,3.300000,17.330000,1.138476e+10,5.253828e+08,5.13000,4.150000,1.960000,11.370000,22.610000,1.234553e+07
Dublin,3.002547e+08,5.710000,11.230000,2.320000,3.280000,22.540000,6.767741e+09,2.736866e+08,5.81000,3.500000,1.920000,13.400000,24.630000,6.665686e+06
Fitzwilliam,3.590351e+08,4.690000,15.000000,2.040000,2.860000,24.590000,8.828673e+09,2.984062e+08,4.96000,3.700000,1.960000,15.140000,25.760000,7.493042e+06
Gilsum,7.612706e+07,5.720000,14.560000,2.430000,3.240000,25.950000,1.975497e+09,7.196517e+07,6.84000,3.460000,1.850000,13.600000,25.750000,1.844162e+06
Greenfield,1.847842e+08,6.380000,11.190000,2.260000,1.090000,20.920000,3.865685e+09,1.702773e+08,8.50000,1.060000,1.960000,15.630000,27.150000,4.596602e+06
Greenville,1.595115e+08,11.300000,6.180000,2.230000,0.880000,20.590000,3.284343e+09,1.203351e+08,12.52000,1.000000,1.930000,7.720000,23.170000,2.705821e+06
